In [1]:
import tensorflow as tf

2022-06-29 11:21:41.327276: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-29 11:21:41.327353: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.7.0


In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaPrevisao"

In [5]:
r = requests.get(url)
j = r.json()
dfTreino = pd.DataFrame.from_dict(j)

In [6]:
dfTreino.head()

,id,grupoAcaoId,periodoTreinoRedeId,tipoExemploTreinoId,regraProjecaoId,redeNeuralId
0,2,12,1,1,30,2


In [7]:
def trataRede(treino):
    print('ID:' , treino['id'])
    nome_arquivo = 'pesos/treino_rede' + str(treino['id']) + '.h5'
    print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(treino['id'])
    json = dfDados.to_dict(orient='list')
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisaoTreino'
    response = requests.post(urlPrevisao, data=json)
    print(response)

In [8]:
urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia"

In [9]:
r = requests.get(urlExemplos)
j = r.json()
dfDados = pd.DataFrame.from_dict(j)

In [10]:
dfDados.head()

,campoX,ticker,diaNumPrevisao
0,"1.137342, 1.127691, 1.125464, 1.133630, 1.134...",ABEV3,20220629
1,"2.271478, 2.281787, 2.329897, 2.271478, 2.288...",AERI3,20220629
2,"1.718131, 1.721686, 1.712544, 1.703911, 1.704...",ALPA4,20220629
3,"1.246201, 1.234043, 1.232219, 1.228571, 1.227...",ALSO3,20220629
4,"1.951087, 1.918478, 1.913043, 1.902174, 1.902...",AMAR3,20220629


In [11]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [12]:
dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

(88, 1800)

In [13]:
dfDados.drop('campoX', inplace=True, axis=1)

In [14]:
dfTreino.apply(trataRede, axis='columns')

ID: 2
pesos/treino_rede2.h5


2022-06-29 11:23:09.721317: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-29 11:23:09.732221: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-06-29 11:23:09.831711: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-29 11:23:11.545898: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12960000 exceeds 10% of free system memory.
2022-06-29 11:23:11.911894: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12960000 exceeds 10% of free system memory.
2022-06-29 11:23:

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1800)              3241800   
                                                                 
 dense_1 (Dense)             (None, 1400)              2521400   
                                                                 
 dense_2 (Dense)             (None, 1000)              1401000   
                                                                 
 dense_3 (Dense)             (None, 600)               600600    
                                                                 
 dense_4 (Dense)             (None, 200)               120200    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 7,885,201
Trainable params: 7,885,201
Non-

0    None
dtype: object